In [62]:
import os
import pytesseract
from pdf2image import convert_from_path, convert_from_bytes
import cv2
import re
import pandas as pd
from datetime import datetime
import pytesseract
import csv
import os
from itertools import zip_longest
import itertools


class Elizabeth:
    
    pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

    def __init__(self):
        self.config = "tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789,"
        self.tesseract_language = "por"
        self.lista_produtos = []
        self.path = 'C:\\Users\\Guilherme\\Pictures\\readerpdf\\pdffiles\\Elizabeth (7).pdf'
        self.imagem = 'C:\\Users\\Guilherme\\Pictures\\readerpdf\\imagens\\'
    
        self.dataatual = str(datetime.today().strftime('%Y-%m-%d %H:%M'))
        self.idmarca = '14'
   
    def converter_imgpdf(self):
        images = convert_from_path(self.path, 200, poppler_path='C:\\Users\\Guilherme\\Pictures\\readerpdf\\poppler-0.68.0\\bin')
        for i in range(len(images)):
            images[i].save(self.imagem+'elizabeth'+ str(i) +'.jpg', 'JPEG')

    def get_referencias(self, sku):

        return str(sku['SKU'].strip())


    def reader_imagem(self):
        imagens = os.listdir(self.imagem)
        search_sku = re.compile(r"\d{7,15}")
        search_saldo = re.compile(r"\d+\.?\d+\,\d+")
        lista_dicts = []
        for imagem in imagens:
            if 'elizabeth' in imagem:
                img = cv2.imread('C:\\Users\\Guilherme\\Pictures\\readerpdf\\imagens\\'+ imagem)
                imagemgray = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                valores = pytesseract.image_to_string(imagemgray, lang=self.tesseract_language, config=self.config)
                valores = valores.split("\n")
                remov_blank = list(filter(lambda k: len(k.strip()) >0, valores))
                sku_produtos = search_sku.findall(str(remov_blank))
                saldo_produtos = search_saldo.findall(str(remov_blank))

                items = list(zip_longest(sku_produtos, saldo_produtos))
                for infos in items:
                    dict_items = {}

                    try:
                        dict_items['SKU'] = str(infos[0].strip().replace("'",""))
                    except:
                        dict_items['SKU'] = 'notfound'
                    try:
                        dict_items['SALDO'] = float(str(infos[1].replace("'","").replace(".","").replace(",",".")))
                    except:
                        dict_items['SALDO'] = float(0)

                    self.lista_produtos.append(dict_items)

        return self.lista_produtos
    
    def group_values(self):
        dicts = self.reader_imagem()
        lista_produtos_final = []
        agrupa_produtos = itertools.groupby(dicts, self.get_referencias)
        for sku, saldos in agrupa_produtos:

            lista = []

            for saldo in saldos:
                try:
                    sku = saldo['SKU']
                    saldo = float(str(saldo['SALDO']).strip().replace(".","").replace(",",".").replace("'",""))
                except:
                    sku = 'notfound'
                    saldo = float(0)
                lista.append(saldo)

            dict_items={

                    'SKU':sku,
                    'SALDO':sum(lista),
                    'IDMARCA':self.idmarca,
                    'DATA':self.dataatual
            }
            lista_produtos_final.append(dict_items)
        print(lista_produtos_final)
                   
                         
elizabeth = Elizabeth()
#elizabeth.converter_imgpdf()
elizabeth.group_values()




[{'SKU': '01010001001471', 'SALDO': 1458.0, 'IDMARCA': '14', 'DATA': '2022-08-12 00:11'}, {'SKU': '01040001001326', 'SALDO': 2280.0, 'IDMARCA': '14', 'DATA': '2022-08-12 00:11'}, {'SKU': '01010001002724', 'SALDO': 10692.0, 'IDMARCA': '14', 'DATA': '2022-08-12 00:11'}, {'SKU': '01010001001616', 'SALDO': 21672.0, 'IDMARCA': '14', 'DATA': '2022-08-12 00:11'}, {'SKU': '01010001001028', 'SALDO': 8748.0, 'IDMARCA': '14', 'DATA': '2022-08-12 00:11'}, {'SKU': '01010001001044', 'SALDO': 20412.0, 'IDMARCA': '14', 'DATA': '2022-08-12 00:11'}, {'SKU': '01010001001060', 'SALDO': 1458.0, 'IDMARCA': '14', 'DATA': '2022-08-12 00:11'}, {'SKU': '01010001001616', 'SALDO': 14448.0, 'IDMARCA': '14', 'DATA': '2022-08-12 00:11'}, {'SKU': '01010001002747', 'SALDO': 13608.0, 'IDMARCA': '14', 'DATA': '2022-08-12 00:11'}, {'SKU': '01040001001326', 'SALDO': 2280.0, 'IDMARCA': '14', 'DATA': '2022-08-12 00:11'}, {'SKU': '01010001001059', 'SALDO': 4374.0, 'IDMARCA': '14', 'DATA': '2022-08-12 00:11'}, {'SKU': '010400